## Setting Sun

Using backtracking to solve the [Setting Sun](https://en.wikipedia.org/wiki/Klotski) puzzle

In [ ]:
import numpy as np

# Sun = 0
# Tall = 1, 2, 4, 5
# Wide = 3
# Small = 6, 7, 8, 9

class SettingSun:
    def __init__(self):
        self.n_rows = 5
        self.n_cols = 4
        self.board = np.zeros(shape=(self.n_rows, self.n_cols))

    def setup(self):
        pieces = [1, 0, 0, 2, 1, 0, 0, 2, -1, 3, 3, -1, 4, 5, 6, 5, 4, 7, 8, 5]
        coords = [(i,j) for i in range(self.n_rows) for j in range(self.n_cols)]

        for p, c in zip(pieces, coords):
            self.board[c] = p

    def get_empty_cells(self):
        """Returns coordinates of empty cells in board"""
        return self.get_piece_coords(-1)
    
    def get_position(self):
        return "".join([str(x) for x in self.board])

    def get_piece_coords(self, piece):
        return [[r, c] for r, c in np.argwhere(self.board == piece)]


In [ ]:
s = SettingSun()
s.setup()
s.board

In [ ]:
coords = s.get_piece_coords(0)

In [ ]:
rows = {x[0] for x in coords}
cols = {x[1] for x in coords}

cols

In [ ]:
l = [(x, min(cols) - 1) for x in rows] # Coordinates to check if trying to move left.
r = [(x, max(cols) + 1) for x in rows] # Coordinates to check if trying to move right.
u = [(min(rows) - 1, x) for x in cols] # Coordinates to check if trying to move up
d = [(max(rows) + 1, x) for x in cols] # Coordinates to check if trying to move down.

In [ ]:
s.board